In [1]:
import sources.logins as lg
from time import sleep
import warnings
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select

from selenium.webdriver.common.alert import Alert
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from selenium import webdriver

from datetime import datetime as dt
warnings.filterwarnings('ignore')

def login_exfm(driver_on = True):
    # generate key using fernet
    # key = Fernet.generate_key().decode()
    key = 'cLAiAdZU1U0sxWWK8sxhF0IWIBP4KrR3xhdu3x1EDQI='
    
    
    # login ExFM and download latest incompleted data
    name,pw = lg.save_id('exfm.txt',key)
    driver = lg.login_url('exfm',name,pw)
    
    
    while True:
        try:
            driver.find_element_by_xpath('//*[@id="RPR_SEARCH_LINK"]').click() # select RMA Search	
            break
        except Exception as e:
            print ('Wrong Username or Password. Try again...')
            name,pw = lg.save_id('exfm.txt',key)
            driver = lg.login_url('exfm',name,pw)
    
    d_type_menu = [
                    'Incompleted',
                    'History',
                    'Equipments',
                    'Customers',
                    'No Download'
                    
                    ]
    #border table
    print(f'\n{"_"*50}')
    print(f'{"|  No.|  Function": <49}|')
    print(f'|{"_"*48}|')
    for i in range(1,1+len(d_type_menu)):
        print(f'|{i: >3}  |  {d_type_menu[i-1]: <40}|')
    print(f'|{"_"*48}|') #bottom border
    while True:
        ind = str(input('Select Dataset to download: '))
        try:
            ind = int(ind)
        except:
            if ind.upper() == 'Q' or ind.upper() == 'QUIT':
                break
            else:
                print('Only accept number')
                continue
        if 0<ind<=len(d_type_menu):
            d_type = d_type_menu[ind-1]
            break
        else:
            print(f'Input number must be less than {len(d_type_menu)}\n')
    
    print(d_type)
    
    # driver.find_element_by_xpath('//*[@id="RPR_SEARCH_LINK"]').click() # select RMA Search
    
    if d_type == d_type_menu[0]: # Incompleted
        driver.find_element_by_xpath('//*[@id="sidEXPORT_CONSOLIDATED_BUTTON_IMAGE"]').click() #click download
    
    if d_type == d_type_menu[1]: # History
        Select(driver.find_element_by_xpath('//*[@id="sidIN_REPAIR_STATUS"]')).select_by_index(0)
        print('Select Status "All" and download.')
        driver.find_element_by_xpath('//*[@id="sidEXPORT_CONSOLIDATED_BUTTON_IMAGE"]').click() #click download
    
    if d_type == d_type_menu[2]: # Equipments
        driver.find_element_by_xpath('//*[@id="EQP_MGT_LINK"]').click() # Equipments
        sleep(0.5)
        driver.find_element_by_xpath('//*[@id="sidEXPORT_BUTTON_IMAGE"]').click()
    
    if d_type == d_type_menu[3]: # Customers
        driver.find_element_by_xpath('//*[@id="ACC_MGT_LINK"]').click() 
        sleep(0.5)
        driver.find_element_by_xpath('//*[@id="sidEXPORT_BUTTON_IMAGE"]').click()
    else:
        print('No Download')

    # check latest file
    i_wait = 0
    while True:
        try: # check file already
            file, ctime = lg.file_latest(folder_name='Downloads')
            if file.endswith('xls'):
                print (f'\nDonwload file {file} succesful at {ctime}')
                break
            else:
                print(f'Please wait for compressing file {file}...{i_wait}s')
                i_wait +=3
                sleep(3)
        except Exception as e: #waiting for download
            if d_type == d_type_menu[-1]: break
            print(f'Please wait for dowloading...{i_wait}s')
            i_wait +=3
            sleep(3)
            
    if driver_on:
        return driver
    else:
        #finish close driver
        driver.close()
    sleep(2)

def auto_accept(driver):
    WebDriverWait(driver, 5).until(EC.alert_is_present())
    driver.switch_to.alert.accept()

In [17]:
def create_RMA(driver,last_rma_tb):
    # click Create RMA
    driver.find_element_by_xpath('//*[@id="RPR_EDIT_LINK"]').click()
    
    # processing last_rma_tb
    rma = last_rma_tb['RMA No.'][0]
    sn = last_rma_tb['SERIAL_NO'][0]
    contact_by = last_rma_tb['Contacted by'][0]
    contact_name = last_rma_tb['Contact Name'][0]
    request = last_rma_tb['Customer Request'][0]
    if contact_name == '': contact_name = '.'
    if str(request) == 'None': request = '.'
    receive = dt.strptime(last_rma_tb['RECIEVE_DATE'][0],'%Y-%m-%d %H:%M:%S').strftime('%d-%b-%y')
    inspect = dt.strptime(last_rma_tb['IN_INSPECT_DATE'][0],'%Y-%m-%d %H:%M:%S').strftime('%d-%b-%y')
    ship_out = dt.strptime(last_rma_tb['Shipped'][0],'%Y-%m-%d %H:%M:%S').strftime('%d-%b-%y')
    note = last_rma_tb['Internal Note'][0]
    # Search Serial
    sn_field = driver.find_element_by_xpath('//*[@id="sidEQP_BODY_NO"]')
    sn_field.clear()
    sn_field.send_keys(sn)
    sn_field.send_keys(Keys.RETURN)

    #Get message
    try:
        msg = driver.find_element_by_xpath('//*[@id="messageLine"]/td[2]/div/span').get_attribute('innerHTML')
    except:
        msg =''
    customer_name = driver.find_element_by_xpath('//*[@id="sidACC_NAME"]').get_attribute('value')
    print(customer_name)

    if customer_name !='':
        if msg =='':
            driver.find_element_by_xpath('//*[@id="sidINFORMATION_GET_MEDIUM_CODE"]').send_keys(contact_by)
            driver.find_element_by_xpath('//*[@id="sidCUSTOMER_PERSON_NAME"]').send_keys(contact_name)
            driver.find_element_by_xpath('//*[@id="sidNOTE_CUSTOMER_COMPLAINT"]').send_keys(request)
            if str(note) !='None':
                history = f'''{note}\n-----History Last RMA# {rma}------------\nReceived:{receive}\nInspection:{inspect}\nShip out: {ship_out}'''
                driver.find_element_by_xpath('//*[@id="sidNOTE_INTERNAL"]').send_keys(history)
            else:
                history = f'''-----History Last RMA# {rma}------------\nReceived:{receive}\nInspection:{inspect}\nShip out: {ship_out}'''
                driver.find_element_by_xpath('//*[@id="sidNOTE_INTERNAL"]').send_keys(history)
        else:
            print(msg)

    # save and printout details
    try:
        bt_save = driver.find_element_by_xpath('//*[@id="sidADD_BUTTON_IMAGE"]')
        bt_save.click()
    except:
        pass
    sleep(0.5)
    # check rma and model
    rma = driver.find_element_by_xpath('//*[@id="sidREPAIR_ID"]').get_attribute('value')
    model = driver.find_element_by_xpath('//*[@id="sidEQP_NAME"]').get_attribute('value')
    create_date = driver.find_element_by_xpath('//*[@id="sidDISP_RPR_STATUS_DATE_1"]').get_attribute('value')
    print('-'*50)
    print('RMA:       ', rma)
    print('Customer:  ',customer_name)
    print('Model:     ',model)
    print('Serial:    ',sn)
    print('Received:  ',create_date)
    print('Request:   ', request)
    print('Note:      ', note,'\n')

def received(drive,sn):
    
        
    #click recieved button
    driver.find_element_by_xpath('//*[@id="sidRPR_STATUS_BUTTON_2"]').click()
    sleep(0.5)
    try:
        driver.find_element_by_xpath('//*[@id="sidCOMPLETE_BUTTON_IMAGE"]').click()
    except:
        print(f'Already completed Receive {sn}')
        driver.find_element_by_xpath('//*[@id="sidBACK_BUTTON_IMAGE"]').click()
        
def basic_page(diver,sn):
    #loading data inspect
    driver.get('https://exfm-asia-app.fujifilm.co.jp/')
    driver.find_element_by_xpath('//*[@id="RPR_SEARCH_LINK"]').click() #click RMA Search
    sn_field = driver.find_element_by_xpath('//*[@id="sidIN_BODY_NO"]')
    sn_field.clear()
    sn_field.send_keys(sn)
    driver.find_element_by_xpath('//*[@id="sidSEARCH_BUTTON_IMAGE"]').click()
    
    status = driver.find_element_by_xpath('//*[@id="search_result[0].sidRS_CURRENT_STATUS_NAME"]').get_attribute('innerHTML')
    #move to each job page
    driver.find_element_by_xpath('//*[@id="search_result[0].sidRS_REPAIR_ID_data"]').click()

    return sn,status

In [3]:
def inspection(driver,last_rma_tb,pf_code):

    #answer 3 questions
    ans_1 = Select(driver.find_element_by_xpath('//*[@id="sidFDA_QUESTION_RESULT_CODE_1"]'))
    ans_1.select_by_index(2)
    ans_2 = Select(driver.find_element_by_xpath('//*[@id="sidFDA_QUESTION_RESULT_CODE_2"]'))
    ans_2.select_by_index(2)
    ans_3 = Select(driver.find_element_by_xpath('//*[@id="sidFDA_QUESTION_RESULT_CODE_3"]'))
    ans_3.select_by_index(2)
    print('\nSay "No" for 3 questions.')
    p_code = driver.find_element_by_xpath('//*[@id="inspection[0].sidDEF_POS_CODE"]').get_attribute('value')
    scope_count = driver.find_element_by_xpath('//*[@id="sidSCOPE_CONNECT_COUNT"]')

    if p_code =='':
        #Repair Type & Evaluation Results
        if str(last_rma_tb['Repair Size'][0])!='None':
            repair_type = driver.find_element_by_xpath('//*[@id="sidREPAIR_SIZE_NAME"]')
            repair_type.send_keys(last_rma_tb['Repair Size'][0])
        try:
            scope_count.send_keys(str(int(last_rma_tb['Scope Connect Count'][0])))
        except:
            print('scope_count empty.')
        results = driver.find_element_by_xpath('//*[@id="sidNOTE_DEFECT"]')
        try: # ver 1.1.1
            results.send_keys(last_rma_tb['Defect Note'][0])
        except:
            print('Empty Defect Note')
        print('\nCompleted fill up Repair Size and Evaluation Results.')

        if len(pf_code)>10:
            driver.find_element_by_xpath('//*[@id="sidACT_ADD_ROWS10_PART"]').click()
            sleep(0.5)
        for i in range(len(pf_code)):
            p_code = '//*[@id="inspection[' + str(i) + '].sidDEF_POS_CODE"]'
            p_code = driver.find_element_by_xpath(p_code)
            p_code.send_keys(pf_code['P_CODE'][i])
            sleep(0.5)

            f_code = '//*[@id="inspection[' + str(i) + '].sidDEF_CASE_CODE"]'
            f_code = driver.find_element_by_xpath(f_code)
            f_code.send_keys(pf_code['F_CODE'][i])
            sleep(0.5)

            cd_code = '//*[@id="inspection[' + str(i) + '].sidDEF_CAUSE_CODE"]'
            cd_code = driver.find_element_by_xpath(cd_code)
            cd_code.send_keys(pf_code['C/D-Code'][i])
            sleep(0.5)

            r_code = '//*[@id="inspection[' + str(i) + '].sidLBR_CODE"]'
            r_code = driver.find_element_by_xpath(r_code)
            r_code.send_keys(pf_code['R_CODE'][i])
            sleep(0.5)
        print('\nCompleted fill up {} lines of P-F-CD-R-Code'.format(i+1))

    else:    
        print('\nAlready fill inspect information')

def inspect_and_parts(sn,driver,conn):
# sn = '6G402K258'
    q = f'''
            SELECT [rma no.],customer_name,repair_status,[status info],[approval],
            [Serial_no],[contacted by],[contact name],[Customer request],
            [recieve_date],
            in_inspect_date,
            shipped,[internal note],
            [scope connect count],
            in_inspect_user_name as INSPECTION,[repair size],[Defect Note]
            FROM consolidated c
            WHERE UPPER(serial_no) = "{sn.upper()}"
            ORDER BY [rma no.] DESC
            LIMIT 1
        '''
    last_rma_tb = pd.read_sql(q,conn)
    last_rma = last_rma_tb['RMA No.'][0]

    # pf_code
    q = f'''SELECT distinct pf.line_no,p_code,f_code,[c/d-code],r_code
            FROM (pf_code pf
            LEFT JOIN cd_code cd ON (pf.[rma no.] = cd.[rma no.] AND pf.line_no = cd.line_no))
            LEFT JOIN repair_code r ON (pf.[rma no.] = r.[rma no.] AND pf.line_no = R.line_no)
            WHERE cd.[rma no.] = "{last_rma}"
        '''
    pf_code = pd.read_sql(q,conn)

    # parts
    q = f'''
            SELECT line_no,part_no,quantity 
            FROM parts
            WHERE [rma no.] = '{last_rma}'
        '''
    parts = pd.read_sql(q,conn)

#     driver.find_element_by_xpath('//*[@id="RPR_SEARCH_LINK"]').click() #click RMA Search
#     sn_field = driver.find_element_by_xpath('//*[@id="sidIN_BODY_NO"]')
#     sn_field.clear()
#     sn_field.send_keys(sn)
#     driver.find_element_by_xpath('//*[@id="sidSEARCH_BUTTON_IMAGE"]').click()

    #move to each job page
#     driver.find_element_by_xpath('//*[@id="search_result[0].sidRS_REPAIR_ID_data"]').click()

    # click inspection
    driver.find_element_by_xpath('//*[@id="sidRPR_STATUS_BUTTON_3"]').click()
    inspection(driver,last_rma_tb,pf_code)

    #save button
    driver.find_element_by_xpath('//*[@id="sidUPDATE_BUTTON_IMAGE"]').click()
    bt_complete = driver.find_element_by_xpath('//*[@id="sidCOMPLETE_BUTTON_IMAGE"]')
    bt_complete.click()

    # edit internal note
    noted = driver.find_element_by_xpath('//*[@id="sidNOTE_INTERNAL"]').get_attribute('value')
    noted_split = noted.split('\nShip')
    noted = f"{noted_split[0]} - {last_rma_tb['INSPECTION'][0]}\nShip{noted_split[1]}"

    internal_note = driver.find_element_by_xpath('//*[@id="sidNOTE_INTERNAL"]')
    internal_note.clear()
    internal_note.send_keys(noted)

    #edit status info
    stt_info = Select(driver.find_element_by_xpath('//*[@id="sidTEMPORARY_STATUS_CODE"]'))
    stt_info.select_by_index(6) # Awaiting contract Billing

    #save button
    driver.find_element_by_xpath('//*[@id="sidUPDATE_BUTTON_IMAGE"]').click()

    #part selection station
    driver.find_element_by_xpath('//*[@id="sidRPR_STATUS_BUTTON_4"]').click()
    part_selection(driver,parts)
    #save button
    driver.find_element_by_xpath('//*[@id="sidUPDATE_BUTTON_IMAGE"]').click()
    sleep(0.5)
    # return basic
    driver.find_element_by_xpath('//*[@id="sidBACK_BUTTON_IMAGE"]').click()

    
def part_selection(driver,parts):

    p_number = driver.find_element_by_xpath('//*[@id="parts[0].sidPRT_CODE"]').get_attribute('value')
    if p_number =='':
        if len(parts)>10:
            driver.find_element_by_xpath('//*[@id="sidACT_ADD_ROWS10_PART"]').click()
            sleep(0.2)
        if len(parts)>20:
            driver.find_element_by_xpath('//*[@id="sidACT_ADD_ROWS10_PART"]').click()
            sleep(0.2)
        for i in range(parts.shape[0]):
            p_number = '//*[@id="parts[' + str(i) + '].sidPRT_CODE"]'
            p_number = driver.find_element_by_xpath(p_number)
            p_number.send_keys(parts['PART_NO'][i])
            sleep(0.5)

            p_qty = '//*[@id="parts[' + str(i) + '].sidPRT_QUANTITY"]'
            p_qty = driver.find_element_by_xpath(p_qty)
            p_qty.clear()
            p_qty.send_keys(int(parts['QUANTITY'][i]))
            sleep(0.5)
        print('\nCompleted fill up {} items to part list.'.format(i+1))

    else: 
        print('\nAlready fill parts infomation')


In [4]:

driver = login_exfm()


Select account:
1   |   thongle
2   |   Nguyen
3   |   FFAP0104
0   |   CREATE NEW ACCOUNT
Select account by number: 1
thongle is selected to login
Removed Downloads


Create new folder: Downloads
Select Chrome Driver and press enter for authorize Certificate...

Typing your ID...
Typing your password...
Login sucessful!


__________________________________________________
|  No.|  Function                                |
|________________________________________________|
|  1  |  Incompleted                             |
|  2  |  History                                 |
|  3  |  Equipments                              |
|  4  |  Customers                               |
|  5  |  No Download                             |
|________________________________________________|
Select Dataset to download: 5
No Download
No Download


In [ ]:
type(driver)

In [ ]:
from sqlite3 import connect
import pandas as pd
conn = connect('history.db')
# file_name = 'files/ACC_TBL_EXPORT_230912.xlsx'
# dealers = pd.read_excel(file_name)
# dealers.to_sql('dealers',conn,index=False,if_exists='replace')

In [5]:
sn_list = '''
2C730K032
2V564K581
1C728K450
4G361A481
2V644K028
1C741K218
2V567K740
1C675K021
2G348K408
1G402K303
4C591A001
1G391K564
LG361K357
1S094A412
2G361K546
1G402K304
1G403K110
1C728K177
2S101K411
5G361K065
1D102K254
KG402K677
2G361K131
1G391K315
4G361K372
2C643K264
1C741K168
3V567K036
1G374K042
LG361K344
1S094A282
1C695K136
1G391K524
2G402K022
5G348A471
1C692K077
1G391K928
1C653K505
2V696K388
KG391K536
5C692K022
2G391K487
8G403K090
1C728K140
2D127K107
1G412K182
KC643K398
7C727K139
KC643K241
8G403K185
KC643K008
1G402K223
2G403K153
8C643K407
3C309K036
SC309A048
1C692K442
LG361K013
3C643K019
5C692K035
1G202K277
1C692K638
1G403K172
1C692K656

'''
sn_list =sn_list.strip().split('\n')

In [ ]:
for sn in sn_list:
    if sn not in list(sn_pending['SERIAL_NO']):
        print (sn)

In [ ]:
conn = connect('history.db')

In [ ]:
for i in range(1):
    sn = str(input('Input: '))
    sn = sn.strip()
    if sn=='':break
    q = f'''
            SELECT [rma no.],customer_name,repair_status,[status info],[approval],
            [Serial_no],[contacted by],[contact name],[Customer request],
            [recieve_date],
            in_inspect_date,
            shipped,[internal note]
            FROM consolidated c
            WHERE UPPER(serial_no) = "{sn.upper()}"
            ORDER BY [rma no.] DESC
            LIMIT 1
        '''
    try:
        last_rma_tb = pd.read_sql(q,conn)
        display(last_rma_tb)
    except Exception as e:
        print(e)
    

In [ ]:
create_RMA(driver,last_rma_tb)
basic_page(driver,sn)
received(driver,sn)

In [ ]:
sn_done = []
sn_failed = []

In [ ]:

for sn in sn_receive:
    q = f'''
            SELECT [rma no.],customer_name,repair_status,[status info],[approval],
            [Serial_no],[contacted by],[contact name],[Customer request],
            [recieve_date],
            in_inspect_date,
            shipped,[internal note]
            FROM consolidated c
            WHERE UPPER(serial_no) = "{sn.upper()}"
            ORDER BY [rma no.] DESC
            LIMIT 1
        '''
    last_rma_tb = pd.read_sql(q,conn)
    try:
        create_RMA(driver,last_rma_tb)
        basic_page(driver,sn)
        received(driver,sn)
        sn_done.append(sn)
    except Exception as e:
        print(e)
        sn_failed.append(sn)

In [ ]:
sn_failed

### Inspection & Part Select

In [ ]:
sn = sn_list[4]
sn_done = []
sn_failed = []


In [ ]:
sn_receive = ['8G391K371']
sn_done =[]
sn_failed = []
sn_check =[]

In [ ]:
i = 1
for sn in sn_receive:
    if sn not in sn_done:
        driver.find_element_by_xpath('//*[@id="RPR_SEARCH_LINK"]').click() #click RMA Search
        sn_field = driver.find_element_by_xpath('//*[@id="sidIN_BODY_NO"]')
        sn_field.clear()
        sn_field.send_keys(sn)

        driver.find_element_by_xpath('//*[@id="sidSEARCH_BUTTON_IMAGE"]').click()
        sleep(0.5)
        status = driver.find_element_by_xpath('//*[@id="search_result[0].sidRS_CURRENT_STATUS_NAME"]').get_attribute('innerHTML')
        if status == 'Receive':
            try:
                print(f'{i}. Start input data for {sn}')
                inspect_and_parts(sn,driver,conn)
                sn_done.append(sn)
                print('------Done!!!-------------')
            except Exception as e:
                print(e, sn)
                sn_failed.append(sn)
        else:
            print(f'{i}. {sn} in status {status}')
            sn_check.append(sn)
        i+=1

In [ ]:
sn_done.append('1G391K869')

In [ ]:

len(sn_done)

In [ ]:
sn_receive ='''
1C692K645
2G391K372
5V644K138
3V567D038
3S094D038
1C692K870
RC361A105
2G348K618
4G361A316
'''
sn_receive = sn_receive.strip().split('\n')

In [ ]:
sn

In [ ]:
sn = '8G391K371'
q = f'''
            SELECT [rma no.],customer_name,repair_status,[status info],[approval],
            [Serial_no],[contacted by],[contact name],[Customer request],
            [recieve_date],
            in_inspect_date,
            shipped,[internal note],
            [scope connect count],
            in_inspect_user_name as INSPECTION,[repair size],[Defect Note]
            FROM consolidated c
            WHERE UPPER(serial_no) = "{sn.upper()}"
            ORDER BY [rma no.] DESC
            LIMIT 1
        '''
last_rma_tb = pd.read_sql(q,conn)
last_rma = last_rma_tb['RMA No.'][0]

# pf_code
q = f'''SELECT distinct pf.line_no,p_code,f_code,[c/d-code],r_code
        FROM (pf_code pf
        LEFT JOIN cd_code cd ON (pf.[rma no.] = cd.[rma no.] AND pf.line_no = cd.line_no))
        LEFT JOIN repair_code r ON (pf.[rma no.] = r.[rma no.] AND pf.line_no = R.line_no)
        WHERE cd.[rma no.] = "{last_rma}"
    '''
pf_code = pd.read_sql(q,conn)

# parts
q = f'''
        SELECT line_no,part_no,quantity 
        FROM parts
        WHERE [rma no.] = '{last_rma}'
    '''
parts = pd.read_sql(q,conn)
pf_code

In [ ]:
part_selection(driver,parts)

In [ ]:
parts

In [ ]:
inspect_and_parts(sn,driver,conn)

### Change location to Hanoi (Ngan account)

In [ ]:
for sn in sn_list:
    driver.find_element_by_xpath('//*[@id="RPR_SEARCH_LINK"]').click() #click RMA Search
    sn_field = driver.find_element_by_xpath('//*[@id="sidIN_BODY_NO"]')
    sn_field.clear()
    sn_field.send_keys(sn)
    driver.find_element_by_xpath('//*[@id="sidSEARCH_BUTTON_IMAGE"]').click()

    #move to each job page
    driver.find_element_by_xpath('//*[@id="search_result[0].sidRS_REPAIR_ID_data"]').click()

     # click save button
    driver.find_element_by_xpath('//*[@id="sidUPDATE_BUTTON_IMAGE"]').click()
   


In [ ]:
sn_list = '''
KC643K398
1C692K645
2G391K372
5V644K138
3V567D038
3S094D038
1C692K870
RC361A105
2G348K618
4G361A316'''
sn_list = tuple(sn_list)

In [ ]:
parts

In [ ]:
cus_no_done

### Change Status Info


In [ ]:
sn_list = '''
3C741K101

1C730K121
SD102A089
2G391K293
1G202K490
2G202K006
7C643K451
KC643K396
3C692K105
5G391K105
1G402K290
6C692K137
1C309K073
5C643K346
3S094K699
1G361K957
2G391K783

'''
sn_list = sn_list.strip().split('\n')


In [11]:
for sn in sn_list:
    basic_page(driver,sn)
    sleep(0.5)
    #edit status info
    stt_info = Select(driver.find_element_by_xpath('//*[@id="sidTEMPORARY_STATUS_CODE"]'))
    stt_info.select_by_index(6) # Awaiting contract Billing

    #save button
    driver.find_element_by_xpath('//*[@id="sidUPDATE_BUTTON_IMAGE"]').click()
    sleep(0.5)
    # complete part select
    parts = driver.find_element_by_xpath('//*[@id="sidDISP_RPR_STATUS_PERSON_4"]').get_attribute('value')
    if parts =='':
        driver.find_element_by_xpath('//*[@id="sidRPR_STATUS_BUTTON_4"]').click()
        driver.find_element_by_xpath('//*[@id="sidCOMPLETE_BUTTON_IMAGE"]').click()
        sleep(0.2)
        try:
            auto_accept(driver)
            print(f'{sn} has parts no ROHS')
        except:
            print(f'Done {sn}')

Done 1C728K450
Done 4G361A481
Done 2V644K028
Done 1C741K218
2V567K740 has parts no ROHS
Done 2G348K408
Done 1G402K303
Done 4C591A001
Done 1G391K564
Done LG361K357
Done 1S094A412
Done 2G361K546
Done 1G402K304
Done 1G403K110
Done 1C728K177
Done 2S101K411
Done 5G361K065
Done 1D102K254
Done KG402K677
Done 2G361K131
Done 1G391K315
Done 4G361K372
Done 2C643K264
Done 1C741K168
3V567K036 has parts no ROHS
Done 1G374K042
Done LG361K344
Done 1S094A282
Done 1C695K136
Done 1G391K524
Done 2G402K022
Done 5G348A471
Done 1C692K077
Done 1G391K928
Done 2V696K388
Done KG391K536
Done 5C692K022
Done 2G391K487
Done 8G403K090
Done 1C728K140
Done 2D127K107
Done 1G412K182
Done KC643K398
Done 7C727K139
Done KC643K241
Done 8G403K185
Done KC643K008
Done 1G402K223
Done 2G403K153
Done 8C643K407
Done 3C309K036
Done SC309A048
Done 1C692K442
Done LG361K013
Done 3C643K019
Done 5C692K035
Done 1G202K277
Done 1C692K638
Done 1G403K172
Done 1C692K656


In [10]:
driver.find_element_by_xpath('//*[@id="sidDISP_RPR_STATUS_PERSON_8"]').get_attribute('value')==''

True

### Approval

In [5]:
sn_list = '''
1G402K303
1G402K304
1G403K110
1C728K177
2S101K411
1C741K168
3V567K036
1S094A282
1C695K136
2G402K022
2G202K019
1C309K265
5G348A471
1C692K077
1G391K928
2V696K388
KG391K536
9G402K358
4G374K157
4G374K154
2S094K543
5C643K347
5C643K343
3S094K544
2G361K188
3S094K980
2S094K540
3S094K015
3S094K526
1G202K277

'''
sn_list = tuple(sn_list.strip().split('\n'))

In [18]:
for sn in sn_list:
#     sn = sn_list[1]    
    sn,status = basic_page(driver,sn)
    if status !='Authorization':
        stt = Select(driver.find_element_by_xpath('//*[@id="sidHOLD_CODE"]'))
        # clear lock QA
        stt.select_by_index(0)
        #status info
        Select(driver.find_element_by_xpath('//*[@id="sidTEMPORARY_STATUS_CODE"]')).select_by_index(0)
        sleep(0.2)
        #save 
        driver.find_element_by_xpath('//*[@id="sidUPDATE_BUTTON_IMAGE"]').click()
        # move to Authorization
        driver.find_element_by_xpath('//*[@id="sidRPR_STATUS_BUTTON_5"]').click()
        sleep(0.2)
        #Set approval
        Select(driver.find_element_by_xpath('//*[@id="sidRPR_COMP_TYPE_CODE"]')).select_by_index(1)
        #save 
        driver.find_element_by_xpath('//*[@id="sidAUTHORIZATION_COMPLETE_BUTTON_IMAGE"]').click()
        print(f'Done for {sn}')
    else:
        print(f'{sn} already Done')

1G402K303 already Done
1G402K304 already Done
Done for 1G403K110
Done for 1C728K177
Done for 2S101K411
1C741K168 already Done
Done for 3V567K036
Done for 1S094A282
Done for 1C695K136
Done for 2G402K022
Done for 2G202K019
Done for 1C309K265
Done for 5G348A471
Done for 1C692K077
Done for 1G391K928
Done for 2V696K388
Done for KG391K536
Done for 9G402K358
Done for 4G374K157
Done for 4G374K154
Done for 2S094K543
Done for 5C643K347
Done for 5C643K343
Done for 3S094K544
Done for 2G361K188
Done for 3S094K980
Done for 2S094K540
Done for 3S094K015
Done for 3S094K526
Done for 1G202K277


'Authorization'

In [ ]:
# driver.find_element_by_xpath()

In [ ]:
# driver.find_element_by_xpath()

In [ ]:
# driver.find_element_by_xpath()

In [ ]:
# driver.find_element_by_xpath()

In [ ]:
# driver.find_element_by_xpath()